## 7.3 Cylindrical Maps
### 7.2.1 Global Plots

In [ ]:
import pandas as pd
import numpy as np
from cartopy import crs as ccrs
import matplotlib.pyplot as plt

In [ ]:
# Options to print figures into notebook/increase size
plt.rcParams.update({'font.size': 16, 'figure.figsize': [12, 6]})

In [ ]:
fires = pd.read_csv("data/VIIRSNDE_global2018312.v1.0.txt")
fires.head()

In [ ]:
fig = plt.figure(figsize=[20,20])

ax = plt.subplot(projection=ccrs.PlateCarree())
ax.coastlines()
ax.set_global()

plt.scatter(fires['Lon'], fires['Lat'])
plt.show()

### 7.2.2 Changing Projections

In [ ]:
fig = plt.figure(figsize=[20,20])

ax = plt.subplot(projection=ccrs.LambertAzimuthalEqualArea())

ax.coastlines()
ax.gridlines()
ax.set_global()

plt.scatter(fires['Lon'], fires['Lat'], transform=ccrs.PlateCarree())
plt.show()

### 7.2.3 Regional Plots

In [ ]:
extent = [-125, -120, 38, 44]

In [ ]:
to_proj = ccrs.PlateCarree()
from_proj = ccrs.PlateCarree()

In [ ]:
lonLabels = np.arange(-180, 180, 1.5)
latLabels = np.arange(-90, 90, 1)

In [ ]:
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib import ticker

In [ ]:
fig = plt.figure()
ax = plt.subplot(projection=to_proj)

ax.coastlines('50m')
ax.set_extent(extent)

frp = ax.scatter(fires['Lon'], fires['Lat'], c=fires['frp(MW)'],
    s=fires['frp(MW)'], transform=from_proj)
cbar = plt.colorbar(frp)
cbar.set_label("Fire Radiative Power (MW)", rotation='vertical')

# 1) Maps the gridlines to the variable gl
gl = ax.gridlines(crs=to_proj, draw_labels=True)

# 2) Adds two attributes to gl, which are xlocator and ylocator
gl.xlocator = ticker.FixedLocator(lonLabels)
gl.ylocator = ticker.FixedLocator(latLabels)

# 3) Changes labels to show degrees North/South and East/West
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

# 4) Removed labels from top and right side
# comment out if you want to include
gl.xlabels_top = False
gl.ylabels_right = False

plt.show()

### 7.2.4 Swath Data

In [ ]:
from netCDF4 import Dataset

fname='data/aod/JRR-AOD_v1r1_npp_s201808091955538_e201808091957180_c201808092049460_thinned.nc'
file_id = Dataset(fname)
file_id.variables.keys()

In [ ]:
print(file_id.variables['AOD550'].shape, 
      file_id.variables['Latitude'].shape,
      file_id.variables['Longitude'].shape)

In [ ]:
aod = file_id.variables['AOD550'][:,:]
lat = file_id.variables['Latitude'][:,:]
lon = file_id.variables['Longitude'][:,:]

In [ ]:
print(file_id.variables['AOD550'].valid_range)

In [ ]:
levs = np.arange(0, 1.8, 0.1)
levs

In [ ]:
fig = plt.figure(figsize=[12, 12])
ax = plt.subplot(projection=ccrs.PlateCarree())

ax.coastlines('50m')

extent = [-125, -120, 38, 44]
ax.set_extent(extent)

x1 = ax.contourf(lon, lat, aod, levs, extend='both')

fig.colorbar(x1, extend='both',
    orientation="horizontal", fraction=0.05)

# Adds the active fire scatter plot on top
ax.scatter(fires['Lon'], fires['Lat'], color='red', s=50)

plt.show()

### 7.2.5 Quality Flag Filtering

In [ ]:
# Import quality flag
quality_flag = file_id.variables['QCAll'][:,:]

# Keep all but the "best" quality using masked arrays
maskHQ = (quality_flag != 0)
aodHQ = np.ma.masked_where(maskHQ, aod)

In [ ]:
file_id.variables

In [ ]:
(aod.count()-aodHQ.count())/aod.count()

In [ ]:
# Top plot
fig = plt.figure()

upper_axis = plt.subplot(2,1,1, projection=ccrs.PlateCarree())
upper_axis.set_title("All Quality")

upper_axis.coastlines('50m')

upper_fig = upper_axis.contourf(lon, lat, aod, levs, extend='both')
fig.colorbar(upper_fig, ax=upper_axis, extend='both')

# Bottom plot
lower_axis = plt.subplot(2,1,2, projection=ccrs.PlateCarree())
lower_axis.set_title("High Quality")

lower_axis.coastlines('50m')

lower_fig = lower_axis.contourf(lon, lat, aodHQ, levs, extend='both')
fig.colorbar(lower_fig, ax=lower_axis, extend='both')

plt.show()

## 7.3 Polar Stereographic Maps

In [ ]:
# Cryo-Sat (ESA satellite/data) source: https://n5eil01u.ecs.nsidc.org/ICEBRIDGE/RDEFT4.001/2020.01.01/
import numpy.ma as ma
import xarray as xr

In [ ]:
fname = "data/RDEFT4_20200131.nc"
ice = xr.open_dataset(fname)

In [ ]:
print(ice['sea_ice_thickness'].min())

In [ ]:
ice_masked = ice.where(ice['sea_ice_thickness'] != -9999.)

In [ ]:
to_proj = ccrs.NorthPolarStereo()
from_proj = ccrs.PlateCarree()

plt.figure(figsize=[10,10])
ax = plt.subplot(projection=to_proj)
ax.coastlines('50m')
ax.gridlines()

ice_plot = ax.pcolormesh(ice['lon'], ice['lat'], ice_masked['sea_ice_thickness'], transform=from_proj)
plt.colorbar(ice_plot)

ax.set_extent([-180, 180, 60, 90], crs=ccrs.PlateCarree())
plt.show()

## 7.4 Geostationary Maps

In [ ]:
fname='data/goes-meso/michael/OR_ABI-L1b-RadM1-M3C13_G16_s20182822019282_e20182822019350_c20182822019384.nc'
file_id = Dataset(fname)
list(file_id.variables)

In [ ]:
file_id.variables['x'][0:10], file_id.variables['y'][0:10]

In [ ]:
proj_var = file_id.variables['goes_imager_projection']
print(proj_var)

In [ ]:
# Define the satellite height and central longitude for plots
# Can vary depending on the geo satellite
sat_height = proj_var.perspective_point_height

# Get the radiance data
rad = file_id.variables['Rad'][:,:]

In [ ]:
# Define the globe projection
semi_major = proj_var.semi_major_axis
semi_minor = proj_var.semi_minor_axis

globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)

In [ ]:
central_lon = proj_var.longitude_of_projection_origin

In [ ]:
crs = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height, globe=globe)

In [ ]:
X = file_id.variables['x'][:] * sat_height
Y = file_id.variables['y'][:] * sat_height

imgExtent = (X.min(), X.max(), Y.min(), Y.max())

In [ ]:
fig = plt.figure(figsize=(10,10))

ax = plt.subplot(projection=crs)
ax.coastlines('10m', color='orange', linewidth=2)

ax.imshow(rad, origin='upper', cmap='gray_r', extent=imgExtent)

plt.show()

In [ ]:
glmfname = 'data/GLM-L2-LCFA_2018_282_20_OR_GLM-L2-LCFA_G16_s20182822000200_e20182822000400_c20182822000427.nc'
file_id_glm = Dataset(glmfname)
file_id_glm.variables.keys()

In [ ]:
glmLon = file_id_glm.variables['event_lon'][:]
glmLat = file_id_glm.variables['event_lat'][:]
area = file_id_glm.variables['event_energy'][:]

glmDF = pd.DataFrame({'lat': glmLat, 'lon': glmLon, 'area': area })

In [ ]:
crs = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height, globe=globe)

from_proj =  ccrs.PlateCarree()
plt.figure(figsize=[10,10])

ax = plt.subplot(projection=crs)
ax.coastlines('10m', color='orange', linewidth=2)
ax.set_extent([-90.0, -82.0, 22.0, 30.0])

# Plot ABI
ax.imshow(rad, origin='upper',cmap='gray_r', extent=imgExtent)

# Add GLM data
plt.scatter(glmDF.lon, glmDF.lat, c=glmDF.area, transform=from_proj,
    s = 300, marker='x')
plt.colorbar(extend='both')

plt.show()

## 7.5 Plotting with OpenDAP

In [ ]:
import cartopy.feature as cfeature
import xarray as xr

In [ ]:
baseURL = 'http://www.esrl.noaa.gov'
catalogURL = '/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
sstID = xr.open_dataset(baseURL+catalogURL)
print(sstID)

In [ ]:
sst = sstID.sst
sst

In [ ]:
mostRecent = len(sst.time.values)-1
recentSST = sst.isel(time=mostRecent)

In [ ]:
sstmin = 0
sstmax = 30

In [ ]:
fig = plt.figure(figsize=[10,5])
ax = plt.subplot(projection=ccrs.Orthographic(-90, 0))
recentSST.plot(cmap=plt.get_cmap('plasma'),
    vmin=sstmin, vmax=sstmax, transform=ccrs.PlateCarree())
ax.coastlines('50m')
plt.show()

In [ ]:
levels = np.arange(sstmin, sstmax, 2)

fig = plt.figure(figsize=[10,5])
ax = plt.subplot(projection=ccrs.Orthographic(-90, 0))
recentSST.plot.contourf(levels=levels, cmap=plt.get_cmap('plasma'), transform=ccrs.PlateCarree())
ax.coastlines('50m')
plt.show()